In [1]:
from pyspark.sql import SparkSession
from pyspark.sql.types import StringType
from pyspark.sql.functions import col
from pyspark.sql.types import StringType, StructType, StructField


In [ ]:
from pyspark.sql import SparkSession
spark = (SparkSession.builder
                .config("spark.jars.packages", "org.apache.spark:spark-sql-kafka-0-10_2.12:3.3.1,org.apache.iceberg:iceberg-spark-runtime-3.5_2.12:1.7.1,org.apache.hadoop:hadoop-aws:3.3.2")
                .config("spark.hadoop.fs.s3a.endpoint", "http://localhost:9000") 
                .config("spark.hadoop.fs.s3a.access.key", "minio") 
                .config("spark.hadoop.fs.s3a.secret.key", "minio123") 
                .config("spark.hadoop.fs.s3a.path.style.access", "true") 
                .config("spark.hadoop.fs.s3a.impl", "org.apache.hadoop.fs.s3a.S3AFileSystem")
                .config("spark.sql.catalog.spark_catalog", "org.apache.iceberg.spark.SparkSessionCatalog")
                .config("spark.sql.catalog.spark_catalog.type", "hadoop")
                .config("spark.sql.catalog.spark_catalog.warehouse", "s3a://commerce/feature/") 
                .config("spark.sql.extensions", "org.apache.iceberg.spark.extensions.IcebergSparkSessionExtensions") 
                .appName("feature transaction")
         .getOrCreate())

In [ ]:
execute_query("""
CREATE TABLE IF NOT EXISTS feature.transaction (
    tx_id INT, -- Unique ID for each transaction
    tx_datetime TIMESTAMP,
    tx_amount FLOAT, -- Transaction amount
    tx_during_weekend BOOLEAN,
    tx_during_night BOOLEAN, 
    customer_id INT, 
    terminal_id INT, 
    customer_nb_tx_1day_window INT, -- Customer's number of transactions in the last 1 day
    customer_avg_amount_1day_window FLOAT, -- Customer's average transaction amount in the last 1 day
    customer_nb_tx_7day_window INT, -- Customer's number of transactions in the last 7 days
    customer_avg_amount_7day_window FLOAT, -- Customer's average transaction amount in the last 7 days
    customer_nb_tx_30day_window INT, -- Customer's number of transactions in the last 30 days
    customer_avg_amount_30day_window FLOAT, -- Customer's average transaction amount in the last 30 days
    terminal_nb_tx_1day_window INT, -- Terminal's number of transactions in the last 1 day
    terminal_risk_1day_window FLOAT, -- Terminal's risk score for the last 1 day
    terminal_nb_tx_7day_window INT, -- Terminal's number of transactions in the last 7 days
    terminal_risk_7day_window FLOAT, -- Terminal's risk score for the last 7 days
    terminal_nb_tx_30day_window INT, -- Terminal's number of transactions in the last 30 days
    terminal_risk_30day_window FLOAT, -- Terminal's risk score for the last 30 days 
    row_created_timestamp TIMESTAMP, 
    row_updated_timestamp TIMESTAMP, 
    FOREIGN KEY (customer_id) REFERENCES feature.customer(customer_id),
    FOREIGN KEY (terminal_id) REFERENCES feature.terminal(terminal_id)
)
USING iceberg
""").show(truncate=False)
